#### <left>Бронников Максим Андреевич</left>

max120199@gmail.com

#   <center>Домашняя работа №9</center>

Ссылка на задание и датасет: https://cloud.mail.ru/public/2BvR/5PVTGs2Ah 

In [21]:
import pymorphy2
import nltk
import ujson as json
import matplotlib.pyplot as plt
import numpy as np
import itertools
from nltk.corpus import stopwords
import gzip
from gensim.models import word2vec
from scipy.spatial.distance import cosine
import math

from datetime import datetime

from collections import Counter

import warnings
warnings.filterwarnings("ignore")

In [22]:
class Document:
    def __init__(self, init_dict):
        self.title = init_dict.get('title', '')
        self.description = init_dict.get('description', '')
        self.url = init_dict.get('url', '')
        self.site = init_dict.get('site', '')
        self.ts = datetime.fromtimestamp(init_dict['ts']) if 'ts' in init_dict else -1
    
    def __str__(self):
        res = ''
        res += 'url : %s\n' % self.url
        res += 'date : %s\n' % self.ts
        res += 'title : %s\n' % self.title
        res += 'description : %s\n' % self.description
        res += 'site : %s\n' % self.site
        return res

In [23]:
fin = gzip.open('dataset_mai.jsonl.gz')
for line in itertools.islice(fin, 1):
    data = json.loads(line.strip())
    print(Document(data))

url : http://bloknot-volzhsky.ru/news/volzhane-mogut-podat-zayavlenie-na-letnie-putevki-
date : 2019-11-30 18:26:10
title : Волжане могут подать заявление на летние путевки для детей
description : С понедельника заявления начинают принимать в МФЦ
site : bloknot-volzhsky.ru



In [24]:
fin = gzip.open('dataset_mai.jsonl.gz')
dataset = []
dataset_test = []
for line in itertools.islice(fin, 10000):
    data = json.loads(line.strip())
    dataset.append(Document(data))

## ДЗ - реализовать поиск похожих документов по текстовым векторам и по word2vec векторам


In [25]:
# Функция из лекции
def split_words_v3(a_text):
    cur_word = ''
    prev_is_alpha = False

    for letter in a_text:
        if  (letter.isalpha() and prev_is_alpha or 
            letter.isdigit() and not prev_is_alpha):
            cur_word += letter
        elif (letter.isalpha() and not prev_is_alpha or
             letter.isdigit() and prev_is_alpha):
            if cur_word: yield cur_word
            cur_word = letter
            prev_is_alpha = not prev_is_alpha
        else:
            if cur_word: yield cur_word
            cur_word = ''
            prev_is_alpha = False
    if cur_word: yield cur_word

In [26]:
# Класс нормализатора на основе лекции
class Normalizer:
    def __init__(self):
        self.morth_cache = {}
        self.morth_analyzer = pymorphy2.MorphAnalyzer()

    def get_norm_word_v3(self, a_word):
        if a_word not in self.morth_cache:
            self.morth_cache[a_word] = self.morth_analyzer.parse(a_word)[0].normal_form
        return self.morth_cache[a_word]

In [27]:
# Функции из лекции
def get_words(word_list):
    return [word for word in word_list if word not in stopwords.words("russian")]

def get_doc_words(a_doc, a_split, normalizer):
    for word in itertools.chain(a_split(a_doc.title), a_split(a_doc.description)):
        yield normalizer.get_norm_word_v3(word)

In [28]:
def doc2tdf(a_doc, word2id, norml):
    res = [0] * len(word2id)
    
    
    a_words = list(get_doc_words(a_doc, split_words_v3, norml))
    for word in a_words: 
        if word in word2id: #проверка для тестового сета
            res[word2id[word]] += 1 
    return res
  #  return list(np.multiply(res, 1 / len(a_words)))

# def set2idf(a_dataset, word2id, norml):
#     idf = [0] * len(word2id)
    
#     for word in word2id:
#         for item in a_dataset:
#             if word in list(get_doc_words(item, split_words_v3, norml)):
#                 idf[word2id[word]] += 1
#         idf[word2id[word]] = np.log(len(a_dataset) / idf[word2id[word]])
    
#     return idf

def set2idf(a_dataset, word2id, norml):
    idf = [0] * len(word2id)
    li = []
    for item in a_dataset:
        for word in list(get_doc_words(item, split_words_v3, norml)):
            if word not in li:
                idf[word2id[word]] += 1
        
        li = []
    for word in word2id:
        idf[word2id[word]] = np.log(len(a_dataset) / idf[word2id[word]])
        
    return idf

### Примечание к реализации

В третьем задании(функции *get_w2v_most_similar_doc*) требуется найти наиболее схожий документ используя усредненный *w2v* вектор. Техника нахождения усредненного взвешенного вектора документа децствительно используется для задач машинного обучения, однако мне не удалось найти примеров применения этой техники для определения схожести двух текстов. поэтому я не придумал никакого более оптимльного варианта применения полученных векторов документов, чем нахождение косинуса между ними и отбор по его наименьшему значению.

In [29]:
def get_word_match_most_similar_docs(a_doc, a_dataset, a_top_n=10):
    # Считем количесто вхождений слов из оригинала в каждом файле и выводим количество наиболее часто совпадпающих
    norml = Normalizer()
    a_words = list(get_doc_words(a_doc, split_words_v3, norml))
    counts = []
    for item in a_dataset:
        count = 0
        temp_list = list(get_doc_words(item, split_words_v3, norml)) 
        for word in a_words:
            if word in temp_list:
                count += 1
        counts.append(count)
    
    # вывоим оригинал
    print("Original doc:")
    print(a_doc)
    # Выводим первые наиболее похожих документа
    
    maximum = max(counts)
    for i, weight in sorted(enumerate(counts), key = lambda x: float(x[1]), reverse = True)[:a_top_n]:
        print("Similary :", weight / maximum)
        print(a_dataset[i])
        

                

def get_tf_idf_most_similar_doc(a_doc, a_dataset, a_top_n=10):
    # для каждого документа строится вектор размерности словаря (аналогично random forest) и добавляется idf
    # далее cosine между документами
    norml = Normalizer()
    word2id = {}
    for item in a_dataset:
        for word in list(get_doc_words(item, split_words_v3, norml)):
            if word not in word2id: word2id[word] = len(word2id)
            
    # Считаем idf    
    idf = set2idf(a_dataset, word2id, norml)
    
    # для каждого вектора расчитываем tf idf 
    v_doc = list(np.multiply(doc2tdf(a_doc, word2id, norml), idf))
    cosines = []
    for item in a_dataset:
        # tf * idf
        v_i = list(np.multiply(doc2tdf(item, word2id, norml), idf))
        if math.isnan(cosine(v_doc, v_i)):
            # Если nan  то выводим в конце списка совпадений
            cosines.append(1.0)
        else:
            cosines.append(cosine(v_doc, v_i))
    
    # вывоим оригинал
    print("Original doc:")
    print(a_doc)
    # Выводим первые наиболее похожих документа
    for i, weight in sorted(enumerate(cosines), key = lambda x: float(x[1]))[:a_top_n]:
        print("Similary :", 1 - weight)
        print(a_dataset[i])
    
    
    
    

def get_w2v_most_similar_doc(a_doc, a_dataset, a_top_n=10):
    # считается средний вектор по всем словам (можно при усреднении учитывать idf)
    norml = Normalizer()
    # Создаем модель work2vec и тренеруем ее на данных их татасета ддя вычисления 
    # векторов для каждого слова в датасете
    w2v = word2vec.Word2Vec([list(get_doc_words(item, split_words_v3, norml)) for item in a_dataset], \
                            min_count=2, size = 500, workers = 4, iter = 10, alpha = 0.1)
    
    word2id = {}
    for item in a_dataset:
        for word in list(get_doc_words(item, split_words_v3, norml)):
            if word not in word2id: word2id[word] = len(word2id)
                
    idf = set2idf(a_dataset, word2id, norml)
    
    # Считаем усредненный вектор для каждого слова документа
    words = list(get_doc_words(a_doc, split_words_v3, norml))
    doc_vec = np.zeros((1, w2v.vector_size)) #w2v[words[0]] - w2v[words[0]] # нулевой вектор
    for word in words:
        if word in w2v.wv.vocab:
            doc_vec += w2v[word] * idf[word2id[word]]
    
    # Вектор косинусов между оригинальным документом и документами из датасета
    cosines = []
    # для каждого документа вычисляем косинус между ним и оригиналом
    for item in a_dataset:
        words = list(get_doc_words(item, split_words_v3, norml))
        full = np.zeros((1, w2v.vector_size)) # w2v[words[0]] - w2v[words[0]]
        for word in words:
            if word in w2v.wv.vocab:
                full += w2v[word] * idf[word2id[word]]
        
        if math.isnan(cosine(doc_vec, full)):
            cosines.append(1.0)
        else:   
            cosines.append(cosine(doc_vec, full))    
        
    # вывоим оригинал
    print("Original doc:")
    print(a_doc)
    # Выводим первые наиболее похожих документа
    for i, weight in sorted(enumerate(cosines), key = lambda x: float(x[1]))[:a_top_n]:
        print("Similary :", 1 - weight)
        print(a_dataset[i])


## Пример формата выдачи

#### Тестовые данные

In [30]:
doc_id = 13 #dota2
doc_id = 1946 #гороскоп
doc_id = 3388 #хоккей
doc_id = 7601 #телефоны

In [31]:
doc_id = 13 
for similart_func in [get_word_match_most_similar_docs,  get_tf_idf_most_similar_doc, get_w2v_most_similar_doc]:
    similart_func(dataset[doc_id], dataset)

Original doc:
url : https://cyber.sports.ru/dota2/1080755627.html
date : 2019-11-30 18:26:39
title : Результаты Parimatch League Dota 2. Virtus.pro победила
description : 30 ноября завершился турнир Parimatch League. В финале Virtus.pro разгромила HellRaisers со счетом 3:0 и заработала 40 тысяч долларов. Лан-финал Parimatch League прошел с 28 по 30 ноября в Москве. 4 команды разыграли 70 тысяч долларов. Результаты команд 1. Virtus.pro2.
site : cyber.sports.ru

Similary : 1.0
url : https://cyber.sports.ru/dota2/1080755627.html
date : 2019-11-30 18:26:39
title : Результаты Parimatch League Dota 2. Virtus.pro победила
description : 30 ноября завершился турнир Parimatch League. В финале Virtus.pro разгромила HellRaisers со счетом 3:0 и заработала 40 тысяч долларов. Лан-финал Parimatch League прошел с 28 по 30 ноября в Москве. 4 команды разыграли 70 тысяч долларов. Результаты команд 1. Virtus.pro2.
site : cyber.sports.ru

Similary : 0.7924528301886793
url : https://cyber.sports.ru/dota2/108

Original doc:
url : https://cyber.sports.ru/dota2/1080755627.html
date : 2019-11-30 18:26:39
title : Результаты Parimatch League Dota 2. Virtus.pro победила
description : 30 ноября завершился турнир Parimatch League. В финале Virtus.pro разгромила HellRaisers со счетом 3:0 и заработала 40 тысяч долларов. Лан-финал Parimatch League прошел с 28 по 30 ноября в Москве. 4 команды разыграли 70 тысяч долларов. Результаты команд 1. Virtus.pro2.
site : cyber.sports.ru

Similary : 1.0
url : https://cyber.sports.ru/dota2/1080755627.html
date : 2019-11-30 18:26:39
title : Результаты Parimatch League Dota 2. Virtus.pro победила
description : 30 ноября завершился турнир Parimatch League. В финале Virtus.pro разгромила HellRaisers со счетом 3:0 и заработала 40 тысяч долларов. Лан-финал Parimatch League прошел с 28 по 30 ноября в Москве. 4 команды разыграли 70 тысяч долларов. Результаты команд 1. Virtus.pro2.
site : cyber.sports.ru

Similary : 0.9724323823912627
url : https://cyber.sports.ru/dota2/108

In [32]:
doc_id = 1946 
for similart_func in [get_word_match_most_similar_docs,  get_tf_idf_most_similar_doc, get_w2v_most_similar_doc]:
    similart_func(dataset[doc_id], dataset)

Original doc:
url : https://www.obozrevatel.com/astro/news/goroskop-na-1-dekabrya-chto-zhdet-lvov-rakov-dev-i-drugie-znaki-zodiaka.htm
date : 2019-11-30 17:07:11
title : Гороскоп на 1 декабря: что ждет Львов, Раков, Дев и другие знаки зодиака
description : Советы астрологов на воскресенье
site : obozrevatel.com

Similary : 1.0
url : https://www.obozrevatel.com/astro/news/goroskop-na-1-dekabrya-chto-zhdet-lvov-rakov-dev-i-drugie-znaki-zodiaka.htm
date : 2019-11-30 17:07:11
title : Гороскоп на 1 декабря: что ждет Львов, Раков, Дев и другие знаки зодиака
description : Советы астрологов на воскресенье
site : obozrevatel.com

Similary : 0.7647058823529411
url : https://mpsh.ru/16034-astrologicheskij-prognoz-na-god-krysy-goroskop-2020-po-znaku-zodiaka-i-godu-rozhdenija.html
date : 2019-11-30 15:02:44
title : Астрологический прогноз на год Крысы — гороскоп 2020 по знаку зодиака и году рождения
description : Предстоящее 12-летие и особенно ближайший год — отличное время для создания семьи, укр

Original doc:
url : https://www.obozrevatel.com/astro/news/goroskop-na-1-dekabrya-chto-zhdet-lvov-rakov-dev-i-drugie-znaki-zodiaka.htm
date : 2019-11-30 17:07:11
title : Гороскоп на 1 декабря: что ждет Львов, Раков, Дев и другие знаки зодиака
description : Советы астрологов на воскресенье
site : obozrevatel.com

Similary : 1.0
url : https://www.obozrevatel.com/astro/news/goroskop-na-1-dekabrya-chto-zhdet-lvov-rakov-dev-i-drugie-znaki-zodiaka.htm
date : 2019-11-30 17:07:11
title : Гороскоп на 1 декабря: что ждет Львов, Раков, Дев и другие знаки зодиака
description : Советы астрологов на воскресенье
site : obozrevatel.com

Similary : 0.4730840322100336
url : https://prochepetsk.ru/news/19614
date : 2019-11-30 15:02:54
title : Романтичный ужин у Раков и шопинг у Водолеев: гороскоп для всех знаков зодиака
description : Гороскоп на 30 ноября и 1 декабря для всех знаков зодиака
site : prochepetsk.ru

Similary : 0.4315835038546696
url : https://progorod43.ru/news/68371
date : 2019-11-30 15:01

In [33]:
doc_id = 3388 
for similart_func in [get_word_match_most_similar_docs,  get_tf_idf_most_similar_doc, get_w2v_most_similar_doc]:
    similart_func(dataset[doc_id], dataset)

Original doc:
url : https://www.tv21.ru/news/2019/11/30/khokkeisty-murmana-proveli-pervyy-domashniy-match-na-stadione-stroitel
date : 2019-11-30 15:31:47
title : Хоккеисты "Мурмана" провели первый домашний матч на стадионе "Строитель"
description : Игра состоялась в рамках открытия нового сезона Суперлиги по хоккею с мячом.
site : tv21.ru

Similary : 1.0
url : https://www.tv21.ru/news/2019/11/30/khokkeisty-murmana-proveli-pervyy-domashniy-match-na-stadione-stroitel
date : 2019-11-30 15:31:47
title : Хоккеисты "Мурмана" провели первый домашний матч на стадионе "Строитель"
description : Игра состоялась в рамках открытия нового сезона Суперлиги по хоккею с мячом.
site : tv21.ru

Similary : 0.8095238095238095
url : https://komiinform.ru/news/189633
date : 2019-11-30 16:06:42
title : "Строитель" опробовал эжвинский лед
description : Сыктывкарский "Строитель" сегодня, 30 ноября, провел первую тренировку на льду стадиона при спортивном комплексе "Бумажник" в Эжве. Тренировка прошла на максима

Original doc:
url : https://www.tv21.ru/news/2019/11/30/khokkeisty-murmana-proveli-pervyy-domashniy-match-na-stadione-stroitel
date : 2019-11-30 15:31:47
title : Хоккеисты "Мурмана" провели первый домашний матч на стадионе "Строитель"
description : Игра состоялась в рамках открытия нового сезона Суперлиги по хоккею с мячом.
site : tv21.ru

Similary : 1.0
url : https://www.tv21.ru/news/2019/11/30/khokkeisty-murmana-proveli-pervyy-domashniy-match-na-stadione-stroitel
date : 2019-11-30 15:31:47
title : Хоккеисты "Мурмана" провели первый домашний матч на стадионе "Строитель"
description : Игра состоялась в рамках открытия нового сезона Суперлиги по хоккею с мячом.
site : tv21.ru

Similary : 0.5057571612748826
url : https://severpost.ru/read/87561
date : 2019-11-30 17:32:40
title : Первый домашний матч «Мурман» сыграл вничью
description : В своем первом домашнем матче «Мурман» не смог одолеть «Волгу». Игра проходила на стадионе «Строитель». В первом тайме мурманчане уступили хоккеистам из..

In [34]:
doc_id = 7601
for similart_func in [get_word_match_most_similar_docs,  get_tf_idf_most_similar_doc, get_w2v_most_similar_doc]:
    similart_func(dataset[doc_id], dataset)

Original doc:
url : https://megaobzor.com/Stala-izvestna-cena-smartfona-Redmi-K30.html
date : 2019-11-30 12:12:16
title : Стала известна цена смартфона Redmi K30
description : Авторитетный искатель утечек Мукул Шарма поделился подробностями о цене смартфона Redmi K30, официальный анонс которого состоится уже 10 декабря. Если верить источнику, аппарат обойдется в 327 долларов, что намного больше 285 долларов, которые ему приписывали ранее. По предварительным данным, Redmi K30 получит аккумулятор ёмкостью 5000 мАч, квадрокамеру с главным датчиком изображения разрешением 60 Мп, дисплей с частотой обновления 120 Гц, двойную фронтальную камеру на 20 Мп и 2 Мп и восьмиядерный процессор Snapdragon 730G.
site : megaobzor.com

Similary : 1.0
url : https://megaobzor.com/Stala-izvestna-cena-smartfona-Redmi-K30.html
date : 2019-11-30 12:12:16
title : Стала известна цена смартфона Redmi K30
description : Авторитетный искатель утечек Мукул Шарма поделился подробностями о цене смартфона Redmi K30, оф

Original doc:
url : https://megaobzor.com/Stala-izvestna-cena-smartfona-Redmi-K30.html
date : 2019-11-30 12:12:16
title : Стала известна цена смартфона Redmi K30
description : Авторитетный искатель утечек Мукул Шарма поделился подробностями о цене смартфона Redmi K30, официальный анонс которого состоится уже 10 декабря. Если верить источнику, аппарат обойдется в 327 долларов, что намного больше 285 долларов, которые ему приписывали ранее. По предварительным данным, Redmi K30 получит аккумулятор ёмкостью 5000 мАч, квадрокамеру с главным датчиком изображения разрешением 60 Мп, дисплей с частотой обновления 120 Гц, двойную фронтальную камеру на 20 Мп и 2 Мп и восьмиядерный процессор Snapdragon 730G.
site : megaobzor.com

Similary : 1.0
url : https://megaobzor.com/Stala-izvestna-cena-smartfona-Redmi-K30.html
date : 2019-11-30 12:12:16
title : Стала известна цена смартфона Redmi K30
description : Авторитетный искатель утечек Мукул Шарма поделился подробностями о цене смартфона Redmi K30, оф

###### <center>Сделано Бронниковым Максимом</center>
###### <center>15.12.2019</center>